In [4]:
import numpy as np
import matplotlib.pyplot as plt
# Assuming the class is in a file named 'SingleLensFitter.py'
from SingleLensFitter import SingleLensFitter

# 1. Load your data
# Create some dummy data for this example
# In a real case, you would load your data from a file
t0_true, u0_true, tE_true = 7500.0, 0.1, 30.0
true_params = [u0_true, t0_true, tE_true]
time = np.linspace(t0_true - 2 * tE_true, t0_true + 2 * tE_true, 500)
tau = (time - t0_true) / tE_true
u = np.sqrt(u0_true**2 + tau**2)
magnification = (u**2 + 2) / (u * np.sqrt(u**2 + 4))
flux = 1000 * magnification + 500  # F_source = 1000, F_blend = 500
flux_err = np.random.normal(0, 20, len(time))  # Add some noise
flux += flux_err
err = np.ones_like(flux) * 20.0

data_dict = {'MyObservatory': (time, flux, err)}

# 2. Set initial parameter guesses: [u0, t0, tE]
initial_params = np.array([0.15, 7505.0, 25.0])

# 3. Initialize the fitter
fitter = SingleLensFitter(data=data_dict, initial_parameters=initial_params)

# 4. (Optional) Customize the model
# Add finite source and limb darkening effects
fitter.add_limb_darkening(gamma=0.5, lrho=-2.5)
# Add a Gaussian Process model for correlated noise
fitter.add_gaussian_process_model()

# 5. Customize fitter settings
fitter.plotprefix = 'ob12345_fit'
fitter.nwalkers = 100
fitter.nsteps_production = 1000 # Reduced for a quick example

# 6. Run the MCMC sampler
# optimize_first=True is recommended to start walkers near the peak likelihood
fitter.sample(optimize_first=True)

# 7. The results are now stored in fitter.p and fitter.samples,
#    and plots have been saved to disk.
print("Fit results:")
print(f"Best-fit parameters (50th percentile): {fitter.p}")

# 8. Display one of the output plots
plt.figure()
img = plt.imread('ob12345_fit-combined-lightcurve.png')
plt.imshow(img)
plt.axis('off')
plt.show()

Using MyObservatory as reference.
Initial parameters: [ 1.500e-01  7.505e+03  2.500e+01 -2.500e+00  5.000e-01  1.000e+00
 -2.000e+00]
ln Prob =  -949289.30083218
Optimising...
Optimized parameters: [ 1.01753229e-01  7.49999909e+03  2.95787420e+01 -2.70329055e+00
  4.43973291e-01  1.40270252e+00 -1.92907410e+00]
ln Prob =  -2195.171647688355
7 7 100
Running burn-in...
ndim, walkers, nsteps, max_iterations: 7 100 50 200
iteration 1 completed


d:\SingleLensFitter\SingleLensFitter.py:574: RuntimeWarning: invalid value encountered in sqrt
  delta_theta = np.pi/(2.0*n)
d:\SingleLensFitter\SingleLensFitter.py:575: RuntimeWarning: invalid value encountered in sqrt
  u1 = uq*np.cos(theta_1d) - np.sqrt(rho**2 - uq**2 * np.sin(theta_1d)**2) + epsilon
d:\SingleLensFitter\SingleLensFitter.py:591: RuntimeWarning: invalid value encountered in sqrt
  integrand = ((r**2+2.0)/np.sqrt(r**2+4.0)) * \
d:\SingleLensFitter\SingleLensFitter.py:577: RuntimeWarning: invalid value encountered in arcsin
  
d:\SingleLensFitter\SingleLensFitter.py:578: RuntimeWarning: invalid value encountered in arcsin
  u1[theta_1d>np.arcsin(rho/uq)] = 0.0


iteration 2 completed
emcee: Exception while calling your likelihood function:
  params: [ 1.01622593e-01  7.49999790e+03  2.95667922e+01 -3.12981488e+00
  6.83435926e-02  1.36421888e+00 -1.79587887e+00]
  args: []
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "c:\Python312\Lib\site-packages\emcee\ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\SingleLensFitter\SingleLensFitter.py", line 380, in lnprob
    lp += self.lnlikelihood()
          ^^^^^^^^^^^^^^^^^^^
  File "d:\SingleLensFitter\SingleLensFitter.py", line 414, in lnlikelihood
    result, lp = self.linear_fit(data_set_name,mag)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\SingleLensFitter\SingleLensFitter.py", line 434, in linear_fit
    C_inv = linalg.inv(C)
            ^^^^^^^^^^^^^
  File "c:\Python312\Lib\site-packages\scipy\linalg\_basic.py", line 959, in inv
    lu, piv, info = getrf(a1, overwrite_a=overwrite_a)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 